In [1]:
import cv2
import numpy as np
import os
from skimage.util import img_as_float
from skimage import io
from matplotlib import pyplot as plt
import matplotlib.path as mplPath
import util
from timeit import default_timer as timer
from datetime import datetime
from skimage.segmentation import mark_boundaries

In [2]:
folderPageImages = 'parzivalDataset'; 
folderGroundThruth = 'groundTruth'
folderUnetData = 'unetParzivalData'
yimg_train = []
yimg_test = []
yimg_validation = []

#### Leyendo las imágenes de los documentos y sus respectivos archivos XML que contienen los segmentos (ground truth) como coordenadas de polígonos


In [3]:
import importlib
importlib.reload(util)
startTime = datetime.now()   
ximg_train,      xgt_train, xnames_train      = util.unetReadPageImagesAndGroundTruth(folderPageImages, folderGroundThruth, 'train', 1)     
ximg_test,       xgt_test, xnames_test       = util.unetReadPageImagesAndGroundTruth(folderPageImages, folderGroundThruth, 'test', 1)
ximg_validation, xgt_validation, xnames_validation = util.unetReadPageImagesAndGroundTruth(folderPageImages, folderGroundThruth, 'validation', 1)
print("Time taken:", datetime.now() - startTime)

Time taken: 0:00:51.151218


### Convirtiendo las imágenes a escala de grises 

In [ ]:
# ximg_train = util.convertToGrayscale(ximg_train)
# ximg_test = util.convertToGrayscale(ximg_test)
# ximg_validation = util.convertToGrayscale(ximg_validation)

In [4]:
from scipy.misc import imsave

def unetSaveImages(arrImages, arrNames, folder) :
    for i in range (len(arrImages)) :
        imsave(folderUnetData + "/" + folder + "/" + "image" + "/" + arrNames[i], arrImages[i])
    

In [5]:
unetSaveImages(ximg_train, xnames_train, 'train')
unetSaveImages(ximg_test, xnames_test, 'test')
unetSaveImages(ximg_validation, xnames_validation, 'validation')

In [8]:
def unetGenerateGroundThruth (arrImg, arrNames, arrGT, folder, factor) :
    for i in range (len(arrImg)) :
        image = arrImg[i]
        imageC = image.copy()
        imageC = util.rescale(imageC, factor, mode='reflect')
        imageC[:,:] = (0,0,0)
        regions = ['page','text','decoration', 'comment']
        listLabels = []

        for k in range(len(regions)) :
            listPolygons = util.groundThruthFindCountourPointsByRegion(
                folderGroundThruth + "/" + folder + "/" + arrGT[i],
                regions[k])

            flagPointProcessed = False    
            for polygon in listPolygons :
                polygon = np.int32([polygon])
                pts = polygon.reshape((-1,1,2))
                if regions[k] == 'text':
                    cv2.fillConvexPoly(imageC,pts,(0,0,1), cv2.LINE_AA)
                elif regions[k] == 'decoration':
                    cv2.fillConvexPoly(imageC,pts,(1,0,0), cv2.LINE_AA)
                elif regions[k] == 'page':
                    cv2.fillConvexPoly(imageC,pts,(1,1,1), cv2.LINE_AA)
                elif regions[k] == 'comment':
                    cv2.fillConvexPoly(imageC,pts,(1, 0.75, 0.8), cv2.LINE_AA)                
        imageC = util.rescale(imageC, factor**-1, mode='reflect')  
        imsave(folderUnetData + "/" + folder + "/" + "label" + "/" + arrNames[i], imageC)


In [13]:
from matplotlib.path import Path
def unetGenerateLabels (arrImg, arrNames, arrGT, folder, factor) :
    arrLabels = []
    for i in range (len(arrImg)) :
        image = arrImg[i]
        regions = ['page','text','decoration', 'comment']
        label = np.ndarray(shape=(image.shape[0],image.shape[1], 5), dtype=float)
        label[:,:] = (0.0, 0.0, 0.0, 0.0, 1.0)
        for k in range(len(regions)) :
            listPolygons = util.groundThruthFindCountourPointsByRegion(
                folderGroundThruth + "/" + folder + "/" + arrGT[i],
                regions[k])

            flagPointProcessed = False    
            for polygon in listPolygons :
                #polygon = np.int32([polygon])
                #pts = polygon.reshape((-1,1,2))
                p = Path(polygon)
                grid = p.contains_points(points)
                if regions[k] == 'text':
                    label[grid] = (1.0, 0.0, 0.0, 0.0, 0.0)
                elif regions[k] == 'decoration':
                    label[grid] = (0.0, 1.0, 0.0, 0.0, 0.0)
                elif regions[k] == 'page':
                    label[grid] = (0.0, 0.0, 1.0, 0.0, 0.0)
                elif regions[k] == 'comment':
                    label[grid] = (0.0, 0.0, 0.0, 1.0, 0.0)
            arrLabels.add(label)
        return arrLabels


In [14]:
arr = unetGenerateLabels(ximg_train, xnames_train, xgt_train,'train', 2**3)

ValueError: 'vertices' must be a 2D list or array with shape Nx2

In [28]:
unetGenerateGroundThruth(ximg_train, xnames_train, xgt_train,'train', 2**3)
unetGenerateGroundThruth(ximg_test, xnames_test, xgt_test, 'test', 2**3)
unetGenerateGroundThruth(ximg_validation, xnames_validation, xgt_validation, 'validation', 2**3)

<b>Pintando imagen original y ground truth</b>

In [ ]:
import importlib
importlib.reload(util)
imageGT = util.paintGroundTruthImage(imgSelectedToShow, gtToShow, folderGroundThruth, 'train', 2**3)

In [ ]:
util.subplot(titles=['original','gt'], images = [imgSelectedToShow, imageGT], rows = 1, imgPerRows = 2, size = 20)


index 7 to samples

In [ ]:
# ximg_train = util.convertToGrayscale(ximg_train)
# ximg_test = util.convertToGrayscale(ximg_test)
# ximg_validation = util.convertToGrayscale(ximg_validation)

### Obteniendo las clases o etiquetas por cada ejemplar de entrenamiento (patch)